In [3]:
vuelos_RDD = sc.textFile("../data-spark/flight-data/airport-codes-na.txt")

In [4]:
vuelos_RDD.count()

527

In [7]:
vuelos_RDD.getNumPartitions()

2

In [8]:
vuelos_RDD = sc.textFile("../data-spark/flight-data/airport-codes-na.txt", minPartitions=4, use_unicode=True)

In [9]:
vuelos_RDD.getNumPartitions()

4

In [11]:
vuelos_RDD.saveAsTextFile('../data-spark/flight-data/output')

In [16]:
from pyspark.sql.types import *

In [20]:
vuelos_proceso_RDD = vuelos_RDD.map(lambda vuelos:vuelos.split())

In [17]:
esquema = StructType([
    StructField("date", DateType(), True),
    StructField("delay", StringType(), True),
    StructField("distance", LongType(), True),
    StructField("origin", StringType(), True),
    StructField("destination", StringType(), True),
])

In [21]:
vuelos_sin_cabecera_RDD = vuelos_proceso_RDD.filter(lambda registro:registro[0] != 'City')

In [22]:
vuelos_sin_cabecera_RDD.take(5)

[['Abbotsford', 'BC', 'Canada', 'YXX'],
 ['Aberdeen', 'SD', 'USA', 'ABR'],
 ['Abilene', 'TX', 'USA', 'ABI'],
 ['Akron', 'OH', 'USA', 'CAK'],
 ['Alamosa', 'CO', 'USA', 'ALS']]

In [23]:
vuelos_proceso_RDD.take(5)

[['City', 'State', 'Country', 'IATA'],
 ['Abbotsford', 'BC', 'Canada', 'YXX'],
 ['Aberdeen', 'SD', 'USA', 'ABR'],
 ['Abilene', 'TX', 'USA', 'ABI'],
 ['Akron', 'OH', 'USA', 'CAK']]

In [24]:
vuelos_ok_RDD = vuelos_sin_cabecera_RDD\
    .map(lambda registro:(registro[3], registro[0]))

In [25]:
vuelos_ok_RDD.take(5)

[('YXX', 'Abbotsford'),
 ('ABR', 'Aberdeen'),
 ('ABI', 'Abilene'),
 ('CAK', 'Akron'),
 ('ALS', 'Alamosa')]

In [26]:
retrasos_RDD = sc.textFile("../data-spark/flight-data/departuredelays.csv")

In [27]:
retrasos_RDD.take(5)

['date,delay,distance,origin,destination',
 '01011245,6,602,ABE,ATL',
 '01020600,-8,369,ABE,DTW',
 '01021245,-2,602,ABE,ATL',
 '01020605,-4,602,ABE,ATL']

In [30]:
retrasos_proceso_RDD = retrasos_RDD.map(lambda vuelos:vuelos.split(","))

In [31]:
retrasos_proceso_RDD.take(5)

[['date', 'delay', 'distance', 'origin', 'destination'],
 ['01011245', '6', '602', 'ABE', 'ATL'],
 ['01020600', '-8', '369', 'ABE', 'DTW'],
 ['01021245', '-2', '602', 'ABE', 'ATL'],
 ['01020605', '-4', '602', 'ABE', 'ATL']]

In [32]:
retrasos_sin_cabecera_RDD = retrasos_proceso_RDD.filter(lambda registro:registro[0] != 'date')

In [33]:
retrasos_sin_cabecera_RDD.take(5)

[['01011245', '6', '602', 'ABE', 'ATL'],
 ['01020600', '-8', '369', 'ABE', 'DTW'],
 ['01021245', '-2', '602', 'ABE', 'ATL'],
 ['01020605', '-4', '602', 'ABE', 'ATL'],
 ['01031245', '-4', '602', 'ABE', 'ATL']]

In [34]:
retrasos_ok_RDD = retrasos_sin_cabecera_RDD\
    .map(lambda registro:(registro[3], registro[1]))

In [35]:
retrasos_ok_RDD.take(5)

[('ABE', '6'), ('ABE', '-8'), ('ABE', '-2'), ('ABE', '-4'), ('ABE', '-4')]

In [36]:
vuelos_ok_RDD.take(5)

[('YXX', 'Abbotsford'),
 ('ABR', 'Aberdeen'),
 ('ABI', 'Abilene'),
 ('CAK', 'Akron'),
 ('ALS', 'Alamosa')]

In [37]:
combinacion_retrasos_vuelos_RDD = retrasos_ok_RDD.join(vuelos_ok_RDD)

In [38]:
combinacion_retrasos_vuelos_RDD.sortByKey(ascending=False).getNumPartitions()

6

In [39]:
combinacion_retrasos_vuelos_RDD.take(5)

[('ABE', ('6', 'Allentown')),
 ('ABE', ('-8', 'Allentown')),
 ('ABE', ('-2', 'Allentown')),
 ('ABE', ('-4', 'Allentown')),
 ('ABE', ('-4', 'Allentown'))]